Installing required resources
----------------

In [ ]:
# Using pip3 to install for Python 3.x; remove the quiet flag (-q) if you want to see the output

# Standard downloads
!pip3 -q install numpy cython scipy sqlalchemy pandas matplotlib pymatbridge --user

# The spinkyDemo package, which provide a set of utilitary functions useful to avoid clutering 
# this example with a lot of code.
!pip3 -q install git+https://github.com/christian-oreilly/spinkyDemo.git --user --upgrade

# Cloning the SPINKY Matlab Git repository
import os
if not os.path.isdir("SPINKY"):
    !git clone https://github.com/TarekLaj/SPINKY.git
        
# Downloading an example of EEG + scoring from the DREAMS database        
!wget "http://www.tcts.fpms.ac.be/~devuyst/Databases/DatabaseSpindles/excerpt1.txt"
!wget "http://www.tcts.fpms.ac.be/~devuyst/Databases/DatabaseSpindles/Visual_scoring1_excerpt1.txt"   

Importing required functions and packages
--------------

In [1]:
# Standard imports
import numpy as np
from scipy.io import savemat 
import sys
import os
from glob import glob
import pandas as pd

# Imports from spinkyDemo
from spinkyDemo import training_process, test_process
from spinkyDemo import sp_thresholds_ranges as thresholds_ranges  # import kp_thresholds_ranges for K-complexes

/home/oreilly/.local/lib/python3.4/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


Defining some constants...
---------------------------

In [8]:
# Constants
spinkyPath      = os.path.join(os.getcwd(), 'SPINKY')
eegFileName     = "excerpt1.txt"
scoringFileName = "Visual_scoring1_excerpt1.txt"
subjectName     = "exerpt1"
fs              = 100.0       # Sampling frequency, in Hz
pageDuration    = 30.0        # Page length, in seconds
mode            = 'spindles'  # use 'kcomplex' for K-complexes
nbTrainPage     = 20          # Number of pages to use for the training  

Preparing the training and testing signals
---------------------

In [3]:
eegSig          = pd.read_csv(eegFileName, names=["signal"], skiprows=[0])["signal"].values
scoring         = pd.read_csv(scoringFileName, sep="\t", names=["start", "duration"], skiprows=[0])
scoringStart    = scoring["start"].values
scoringDuration = scoring["duration"].values
samplePerPage   = int(pageDuration*fs)
numberOfPages   = int(len(eegSig)/samplePerPage)

# Getting signals based on two mutually exclusive set of pages for training and testing
pageIds      = range(numberOfPages)
trainPageIds = np.random.choice(pageIds, nbTrainPage, replace=False)
trainSig     = eegSig[np.concatenate([np.arange(samplePerPage)+i*samplePerPage 
                                      for i in pageIds if i in trainPageIds])]
testSig      = eegSig[np.concatenate([np.arange(samplePerPage)+i*samplePerPage 
                                      for i in pageIds if not i in trainPageIds])]

# Calculating the number of spindles scored in the training pages
def eventInPage(eventStart, pageId):
    pageStart = pageId*pageDuration
    pageEnd   = pageStart + pageDuration    
    return (eventStart >= pageStart)*(eventStart < pageEnd)

nbEvents = [sum(eventInPage(scoringStart, pageId)) for pageId in trainPageIds]

Starting Matlab and adding necessary paths to Matlab environement variable
--------

In [4]:
from pymatbridge import Matlab
mlab = Matlab(maxtime=60) #Matlab(matlab='C:/Program Files/MATLAB/R2015a/bin/matlab.exe')
mlab.start()

matlabCode = \
"rootPath = '" + spinkyPath + "/'; \n" + """
addpath(rootPath) 
addpath([rootPath 'functions'])
addpath([rootPath 'functions/tqwt_matlab_toolbox/tqwt_matlab_toolbox'])
"""
result = mlab.run_code(matlabCode)

Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-9b33ca7b-6f93-4b49-9071-c6256ce5c4cc
Send 'exit' command to kill the server
......MATLAB started and connected!


Example of threshold estimation (for spindles detection)
---------------

In [ ]:
threshold = thresholds_ranges(mlab, trainSig, samplePerPage, fs)[0]

In [ ]:
threshold = training_process(mlab, trainSig, fs, samplePerPage, mode, threshold, nbEvents)
print("Threshold:", threshold) 

Example of spindle detection
---------------

In [6]:
# This call create a file named score_auto_spindles_XXXXXX.txt where XXXXXXX is the value of subjectName
outputFileName = "score_auto_spindles_" + subjectName + ".txt"
test_process(mlab, testSig, fs, samplePerPage, subjectName, threshold, mode)

[[ 0.  0.  2.  3.  0.  0.  2.  2.  1.  3.  2.  1.  0.  2.  2.  0.  2.  3.
   4.  3.  2.  4.  4.  2.  3.  3.  4.  4.  3.  2.  3.  4.  3.  2.  3.  5.
   2.  4.  1.  2.]]


/home/oreilly/.local/lib/python3.4/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Example of performance assessment
---------------

In [ ]:
def makeIndexSignal(spindles, epoch_length, fs):    
    index = np.zeros((int(epoch_length*fs), 1))
    for s, e in spindles:
        index[int(np.round(s*fs)):int(np.round(e*fs))] = 1
    return index
    
    
def compute2classComp(indGold, indTest):    
    
    TP = np.logical_and(indGold, indTest)
    TN = np.logical_and(np.logical_not(indGold), np.logical_not(indTest)) 
    FP = np.logical_and(np.logical_not(indGold), indTest)
    FN = np.logical_and(indGold, np.logical_not(indTest))

    return TP, TN, FP, FN


def prinResults(threshold, percentile, subject, scorer, TP, TN, FP, FN):
    res = (subject, percentile, threshold, scorer,
            sensitivity(TP, TN, FP, FN), 
            specificity(TP, TN, FP, FN),
            PPV(TP, TN, FP, FN), 
            MCC(TP, TN, FP, FN),
            cohenk(TP, TN, FP, FN), 
            F1(TP, TN, FP, FN))
    print(("Subject:%s, percentile:%s, threshold:%s, scorer:%s, "\
           "sensitivity=%f, specificity=%f, PPV=%f,  MCC=%f, CohenK=%f, F1=%f" % res))     
    return res

spindles = [e for e in reader.events if e.name == scorer]

detectedSpindles = readDetectorOutput(resFileName)

TP = []
TN = [] 
FP = []
FN = []
for noPage in np.unique(detectedSpindles["page"]):
    p = pages[noPage-1]
    spindlePage = [(s.startTime - p.startTime, s.startTime + s.duration() - p.startTime) for s in spindles 
                                   if spindleInPage(s, p)]
    spindlePage = []
    for s in spindles:
        if spindleInPage(s, p):
            sig = reader.read([channel], s.startTime,  s.duration())[channel].signal
            pic = s.startTime - p.startTime + float(np.argmin(sig))/fs
            spindlePage.append((pic-0.1, pic+1.3))

    detectSpindlePage = [(l["time"]-0.1, l["time"]+1.3) for index, l 
                             in detectedSpindles[detectedSpindles["page"] == noPage].iterrows()]

    indGold = makeIndexSignal(spindlePage, epoch_length, fs)
    indTest = makeIndexSignal(detectSpindlePage, epoch_length, fs)

    TPtmp, TNtmp, FPtmp, FNtmp = compute2classComp(indGold, indTest)
    TP.extend(TPtmp)
    TN.extend(TNtmp)
    FP.extend(FPtmp)
    FN.extend(FNtmp)

prinResults(threshold, percentile, subject, scorer,
            np.sum(TP, dtype=np.int64), 
            np.sum(TN, dtype=np.int64), 
            np.sum(FP, dtype=np.int64), 
            np.sum(FN, dtype=np.int64))

Closing Matlab
-----------------------

In [ ]:
mlab.quit()